In [1]:
import pandas as pd

In [2]:
cols = ["frame", "x", "y", "obj","size","seq","tbd1","tbd2","tbd3","filename","start","path_time","delta_time","tbd4"]
useful_cols = ["frame", "x", "y", "obj","size","seq","filename","start","path_time","delta_time"]

In [3]:
%time points = pd.read_csv("data/fixed.csv",names=cols,usecols=useful_cols,parse_dates=['start'])

Wall time: 16.8 s


In [4]:
points.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997978 entries, 0 to 2997977
Data columns (total 10 columns):
frame         int64
x             int64
y             int64
obj           int64
size          int64
seq           int64
filename      object
start         datetime64[ns]
path_time     object
delta_time    object
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 554.6 MB


In [ ]:
# points.info()
# len(points)
# points.describe()
points.head(50)

In [6]:
p = points
p.groupby(p.start).size()

start
2017-08-17 01:27:09      7311
2017-08-17 01:42:25      9902
2017-08-17 01:57:38    120780
2017-08-17 02:12:50      5034
2017-08-17 02:28:03      4961
2017-08-17 02:43:14      4042
2017-08-17 02:58:24      9170
2017-08-17 03:13:37      6694
2017-08-17 03:28:49      6448
2017-08-17 03:44:03      2371
2017-08-17 03:59:13      2392
2017-08-17 04:14:23      2465
2017-08-17 04:29:40      2730
2017-08-17 04:45:12      2562
2017-08-17 05:00:24       627
2017-08-17 05:15:36      2908
2017-08-17 05:30:48      1728
2017-08-17 05:46:01      1619
2017-08-17 06:01:15       330
2017-08-17 06:16:30       712
2017-08-17 06:31:49       503
2017-08-17 06:47:02         5
2017-08-17 07:02:14         9
2017-08-17 07:17:25         3
2017-08-17 07:32:37       859
2017-08-17 07:48:15       808
2017-08-17 08:03:27         5
2017-08-17 08:18:51         5
2017-08-17 08:34:05         4
2017-08-17 08:49:19         3
                        ...  
2017-08-24 08:18:53      5519
2017-08-24 08:33:08     26717
2017

In [7]:
for dtype in ['int64','object']:
    selected_dtype = p.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for int64 columns: 19.61 MB
Average memory usage for object columns: 98.61 MB


In [8]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [9]:
p_int = points.select_dtypes(include=['int64'])
converted_int = p_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(p_int))
print(mem_usage(converted_int))

137.24 MB
40.03 MB


In [10]:
compare_ints = pd.concat([p_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

,before,after
uint16,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [12]:
optimized_p = p.copy()

optimized_p[converted_int.columns] = converted_int

print(mem_usage(p))
print(mem_usage(optimized_p))

554.57 MB
457.36 MB


In [13]:
dtypes = optimized_p.dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint

preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'delta_time': 'object',
    'filename': 'object',
    'frame': 'uint16',
    'obj': 'uint16',
    'path_time': 'object',
    'seq': 'uint16',
    'size': 'uint32',
    'start': 'datetime64[ns]',
    'x': 'uint16',
    'y': 'uint16'}


In [ ]:
p_obj = p.select_dtypes(include=['object']).copy()
p_obj.describe()

In [ ]:
fn = p_obj.filename
fn_cat = fn.astype('category')
print(fn_cat.head())


In [ ]:
fn_cat.head().cat.codes

In [ ]:
print(mem_usage(fn))
print(mem_usage(fn_cat))


In [ ]:
converted_obj = pd.DataFrame()

for col in p_obj.columns:
    num_unique_values = len(p_obj[col].unique())
    num_total_values = len(p_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = p_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = p_obj[col]

In [ ]:
optimized_p[converted_obj.columns] = converted_obj

mem_usage(optimized_p)

In [ ]:
print(mem_usage(p_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([p_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

In [ ]:
converted_obj.info(memory_usage='deep')

In [ ]:
dtypes = optimized_p.dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint

preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

In [20]:
cols_types = dict({   
    'delta_time': 'object',
    'filename': 'category',
    'frame': 'uint16',
    'obj': 'uint16',
    'path_time': 'object',
    'seq': 'uint16',
    'size': 'uint32',
    'x': 'uint16',
    'y': 'uint16'})

In [22]:
%%time 
read_and_optimized = pd.read_csv('data/fixed.csv',names=cols,usecols=useful_cols,dtype=cols_types,parse_dates=['start'],infer_datetime_format=True)

print(mem_usage(read_and_optimized))
read_and_optimized.head()

283.10 MB
Wall time: 25.9 s
